Demonstrate an early approach of predicting whether a generated integer is odd/even by viewing the preceeding bits

In [10]:
import numpy as np
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [16]:
# Class used to access my implmentations of PRNGs
class PRNGManagement():
  # Initialises object and sets the default seed
  def __init__(self):
      self.seed = 0

  # Method to seed the PRNG (seeds all PRNGs in class)
  def seed_PRNG(self, seed:int):
    self.seed:int = seed
    self.random_number:int = seed

  def bit_success(self, model, inputData, trueOutputs, sequence_length):
    """ 
    Method to evaluate the provided model and store the amount of
    successful predictions for each bit of the output
    :param model: keras model - Model used to generate predicitions
    :param trueOutputs: list[list[int]]- List containing the expected y outputs
    :param sequence_length: int - Length of generated binary string being predicited
    :return list[int] - Amount of successful predictions for each bit
    """
    # Set initial amount of successful predictions for each bit to zero
    successfulPredicts = [0]*sequence_length
    # Feeds the input data to the model and stores the predictions made
    predicted = (model.predict(inputData).round())
    # Iterate over all outputed data
    for testIndex in range(0, len(inputData)):
      # Iterate over each bit in output
      for i in range(sequence_length):
        # If the predicted bit matches the true bit value then increment the successful predicts for the current bit
        if predicted[testIndex][i] == trueOutputs[testIndex][i]: successfulPredicts[i] += 1
        # Prediction may be greater than 1 if the prediction is made with high certainity
        elif predicted[testIndex][i] > 1 and trueOutputs[testIndex][i] == 1: self.successfulPredicts[i] += 1

    return successfulPredicts
   

  def zero_only_PRNG(self, length=100):
    """ 
    Returns a binary string containing only 0 of specified length.
    Used to test for major flaws in models
    :param length: int - Length of generated binary string
    :return string - generated binary string
    """
    return "0" * length        


  def alternating_bits_PRNG(self, length=100):
    """ 
    Returns output of a basic PRNG implementation that alernates each bit (010101)
    :param length: int - Length of generated binary string
    :return string - generated binary string
    """
    # Use seed to determine the starting bit of the generated binary string
    self.seed = self.seed%2
    # Utilises efficent method to repeat a string pattern
    if (self.seed == 1):
        output = "10" * int(length/2)
    else:
        output = "01" * int(length/2)

    # Length of generated binary string is odd
    if (length%2 == 1):
      # Add final bit to string
      output += str(self.seed)
      # Set the new seed value
      if (self.seed == 0): self.seed = 1
      else: self.seed = 0
    
    return output


  def alternating_num_PRNG(self):
    """ 
    Returns output of a basic PRNG implementation that alernates between two binary strings
    :return string - generated binary string
    """
    # Use seed to determine the binary string to be returned
    self.seed = (self.seed+1)%2
    if (self.seed == 0):
      # Convert integer to a binary string 
      randomBinary = str(bin(1643712566))[2:]
      # Returns binary string after ensuring a minimum length of 32
      return (32-len(randomBinary))*"0" + randomBinary
    else:
      # Convert integer to a binary string 
      randomBinary = str(bin(2372817037))[2:]
      # Returns binary string after ensuring a minimum length of 32
      return (32-len(randomBinary))*"0" + randomBinary


  def basic_equation_based(self, mult:int, add:int, mod:int, leng:int) -> str:
    """ 
    Returns output of a very weak equation based PRNG implementation
    Expected to be predicted near perfectly
    :return string - generated binary string
    """
    # Generates random number using previous output as seed
    self.random_number = (mult * self.random_number + add) % 2**mod
    # Converts generated number to a binary string
    bits_string = bin(self.random_number)[2:]
    # Returns binary string after using padding to ensure a length of 32
    return bits_string.zfill(leng)


  ## Different implmentations of equation based generators
  def basic_equation_based1(self) -> str:
    return self.basic_equation_based(20, 52, 32, 32)

  def basic_equation_based2(self) -> str:
    return self.basic_equation_based(36791, 83247, 32, 32)

  # Expects odd starting seed
  def poor_equation_based(self) -> str:
    return self.basic_equation_based(65539, 0, 31, 32)


PRNGHandler = PRNGManagement()

In [20]:
currentGenerator = PRNGHandler.basic_equation_based2

In [24]:
# Sets paramemters for generating train/test data
num_samples = 1000
sequence_length = 32
# Percentage of data used for testing the created prediction model
testDataPerc = 0.25

In [25]:
# Seeds generator
PRNGHandler.seed_PRNG(23)

# Produces training data
visibleOutput = []
predictionBool = []
for _ in range(num_samples):
  fullOutput = currentGenerator()
  visibleOutput.append([int(bit) for bit in fullOutput[:-1]])
  predictionBool.append([int(bit) for bit in fullOutput[-1:]])

In [ ]:
# Splits the x/y data into test and train data
x_train, x_test, y_train, y_test = train_test_split(visibleOutput, predictionBool, test_size = testDataPerc, random_state = 0)

In [ ]:
# Sets the model type and sets the layers
model = Sequential()
model.add(LSTM(32, input_shape=(sequence_length-1, 1), return_sequences=True))
model.add(LSTM(16, activation='relu',  return_sequences=True))
model.add(LSTM(8, activation='relu', return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

# Compiles the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

High accuracy was only achieved on basic models with highly predictly final bits

In [27]:
# Train the model with the x/y train data
model.fit(x_train, y_train, epochs=10, batch_size=32)

# Evaluates the model using the x/y test data
loss, accuracy = model.evaluate(x_test, y_test)

Epoch 1/10
24/24 [==============================] - 48s 110ms/step - loss: 0.6935 - accuracy: 0.5013
Epoch 2/10
24/24 [==============================] - 3s 111ms/step - loss: 0.6918 - accuracy: 0.5867
Epoch 3/10
24/24 [==============================] - 4s 148ms/step - loss: 0.6908 - accuracy: 0.5747
Epoch 4/10
24/24 [==============================] - 3s 112ms/step - loss: 0.6883 - accuracy: 0.6547
Epoch 5/10
24/24 [==============================] - 3s 111ms/step - loss: 0.6825 - accuracy: 0.5733
Epoch 6/10
24/24 [==============================] - 3s 111ms/step - loss: 0.6516 - accuracy: 0.6653
Epoch 7/10
24/24 [==============================] - 3s 119ms/step - loss: 0.4198 - accuracy: 0.8267
Epoch 8/10
24/24 [==============================] - 4s 156ms/step - loss: 0.1204 - accuracy: 0.9947
Epoch 9/10
24/24 [==============================] - 3s 137ms/step - loss: 0.0244 - accuracy: 1.0000
Epoch 10/10
8/8 [==============================] - 1s 20ms/step - loss: 0.1133 - accuracy: 1.0000
